# 14장 합성곱신경망: 컴퓨터비전 (2부)

#### 감사의 글

자료를 공개한 저자 오렐리앙 제롱과 강의준비에 필요한 자료를 지원한 한빛아카데미에게 진심어린 감사를 전합니다.

## ResNet-34 CNN 구현

* 앞서 소개된 대부분의 CNN 모델들을 케라스가 기본으로 지원함.

* 하지만 실제로 케라스로 구현하는 것도 대부분 매우 쉬움.

### 잔차 유닛(RU) 구현

* ResNet-34 모델에 사용되는 ResidualUnit 층을 직접 구현하는 것도 간단함.

<div align="center"><img src="images/ch14/homl14-23b.png" width="450"/></div>

* ResidualUnit 클래스 구성 요소
    * `main_layers`: 오른쪽 모듈
    * `skip_layers`: 왼쪽 모듈. 보폭이 1보다 큰 경우에만 합성곱 모델 적용.

```python
DefaultConv2D = partial(keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]
```

```python
    def call(self, inputs):
        # main_layers
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)

        # skip_layers
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)
```

### ResNet-34 구현

* Sequential 클래스 활용
* 잔차 유닛을 하나의 층으로 취급 가능

```python
model = keras.models.Sequential()
model.add(DefaultConv2D(64, kernel_size=7, strides=2,
                        input_shape=[224, 224, 3]))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="SAME"))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(10, activation="softmax"))
```

## 케라스 제공 사전훈련된 모델 활용

* 많은 모델이 `keras.applications` 에서 기본제공됨.
    * ResNet 모델 변종
    * Inception-v3, Xception 등 GoogLeNet 모델 변종
    * VGGNet 모델 변종
    * MobileNet, MobileNetV2 등 모바일 전용 모델

### 예제: ResNet-50 모델

#### 모델 불러오기

```python
model = keras.applications.resnet50.ResNet50(weights="imagenet")
```

* `224x224` 모양의 이미지를 입력값으로 사용해야 함.

#### 주의사항: 입력 이미지 모양

* 모델에 따라 입력 이미지의 모양이 달리짐.

* 입력모양 변경: `tf.image.resize()`, `tf.image.crop_and_resize()` 등 다양한 함수 이용 가능.
    * 일반적으로 가로, 세로 비율을 유지하지는 않음.

```python
images_resized = tf.image.resize(images, [224, 224])

또는

images_resized = tf.image.crop_and_resize(images, [china_box, flower_box], 
                                          [0, 1], [224, 224])
```

#### 주의사항: 입력 데이터 값

* 모델에 따라 입력 이미지에 사용되는 값는 0에서1 또는 -1에서 1 사이로 스케일링 된 것을 기대.

* 모델 마다 `preprocess_input()` 제공. 
    * 이미지에 사용된 값이 0에서 255 사이인 것을 기대
    * 예를 들어, 이미지에 사용된 값이 0에서 1사이의 값이라면, 255를 곱해서 입력해 주어야 함.

```python
inputs = keras.applications.resnet50.preprocess_input(images_resized * 255)
```

#### 예측

```python
Y_proba = model.predict(inputs)
```

#### 예측결과 확인

* `decode_predictions()` 메서드 활용
    * 이미지별로 지정된 수 만큼의 최상위 예측 클래스를 보여줌.

* 아래 코드는 두 이미지 각각에 대한 최상위 3개의 클래스를 보여줌.
    * 클래스 수: 1,000개

```python
top_K = keras.applications.resnet50.decode_predictions(Y_proba, top=3)
for image_index in range(len(images)):
    print("Image #{}".format(image_index))
    for class_id, name, y_proba in top_K[image_index]:
        print("  {} - {:12s} {:.2f}%".format(class_id, name, y_proba * 100))
    print()
```

* 결과
    * 정확하지는 않지만 사원(monastery), 데이지(daisy) 등 유사한 클래스가 탑 3 항목에 포함되었음.

```python
Image #0
  n03877845 - palace       43.39%
  n02825657 - bell_cote    43.07%
  n03781244 - monastery    11.70%

Image #1
  n04522168 - vase         53.96%
  n07930864 - cup          9.52%
  n11939491 - daisy        4.97%
```

## 사전훈련된 모델 활용 전이학습

* 이미지넷에 없는 이미지 클래스를 감지하는 분류기를 만들고자 하는 경우 활용

### 예제: 사전훈련된 Xception 모델 활용

* 사전훈련된 Xception 모델을 활용한 꽃 이미지 분류기 모델 훈련하기

#### 데이터 불러오기

* `tensorflow_datasets` 모듈의 `load()` 함수 활용하여 샘플 데이터셋 불러오기

```python
import tensorflow_datasets as tfds

dataset, info = tfds.load("tf_flowers", as_supervised=True, with_info=True)
```

* `tf_flowers` 데이터셋
    * 훈련 세트만 존재하는 데이터셋
    * 5개의 꽃 클래스: `['dandelion', 'daisy', 'tulips', 'sunflowers', 'roses']`
    * 샘플 수: 3,670

* 테스트 세트(10%), 검증 세트(15%), 훈련 세트(75%)로 분리하기

```python
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
    as_supervised=True)
```

#### 전처리

* 배치 활용: 크기는 32
* 섞기(shuffle) 실행
* `randomize=True`: 무작위적으로 사진자르기와 수평뒤집기 등을 활용한 데이터 증식
* `224x224` 모양으로 변환 후 `preprocess_input()` 메서드로 전치리 실행
* `prefetch(1)`: 배치 미리 준비시키기

```python
def preprocess(image, label, randomize=False):
    if randomize:
        cropped_image = random_crop(image)
        cropped_image = tf.image.random_flip_left_right(cropped_image)
    else:
        cropped_image = central_crop(image)
    resized_image = tf.image.resize(cropped_image, [224, 224])
    final_image = keras.applications.xception.preprocess_input(resized_image)
    return final_image, label

batch_size = 32
train_set = train_set_raw.shuffle(1000).repeat()
train_set = train_set.map(partial(preprocess, randomize=True)).batch(batch_size).prefetch(1)
valid_set = valid_set_raw.map(preprocess).batch(batch_size).prefetch(1)
test_set = test_set_raw.map(preprocess).batch(batch_size).prefetch(1)
```

#### ImageNet에서 사전훈련된 Xception 모델 활용

* 분리 합성곱 층들을 제외한 (최상위에 위치한) 층 제거
    * 전역평균 층
    * 밀집 출력층

<div align="center"><img src="images/ch14/homl14-24d.png" width="700"/></div>

<그림 출처: [Xception: Deep Learning with Depthwise Separable Convolutions](https://arxiv.org/pdf/1610.02357.pdf)>

* 최상위 몇 개의 층 제거 후 새로은 층 두 개 추가
    * 전역평균층
    * softmax 활성화함수를 사용하는 밀집 출력층

```python
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(n_classes, activation="softmax")(avg)
model = keras.models.Model(inputs=base_model.input, outputs=output)
```

#### 1차 훈련

* 사전훈련된 층의 가중치를 동결 후 훈련
* 학습률: `lr=0.2`
* 성능: 검증 세트에 대한 정확도가 88% 정도

```python
# 사전훈련된 층의 가중치 동결하기
for layer in base_model.layers:
    layer.trainable = False

# 컴파일 후 훈련 시작
optimizer = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=5)
```

#### 2차 훈련

* 사전훈련된 층의 가중치 동결 해제 후 다시 훈련
* 학습률 작게: `lr=0.0.01`
* 성능: 검증 세트에 대한 정확도가 94.3% 정도

```python
# 사전훈련된 층의 가중치 동결 해제
for layer in base_model.layers:
    layer.trainable = True

# 재 컴파일 후 훈련 시작
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9,
                                 nesterov=True, decay=0.001)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(train_set,
                    steps_per_epoch=int(0.75 * dataset_size / batch_size),
                    validation_data=valid_set,
                    validation_steps=int(0.15 * dataset_size / batch_size),
                    epochs=40)
```

## 분류와 위치추정(classification and localization)

* 사진에 포함된 꽃을 분류하면서 해당 꽃의 위치 추청하기

* 위치추정은 회귀 모델로 구현 가능
    * 탐색대상인 객체의 주위로 네모 모양의 __경계상자__(bounding box) 그리기
    * 네모상자의 중심좌표와 높이와 너비(세로와 가로), 즉, 네 개의 값을 예측해야 함.

### 경계상자 추정 모델 구현

* 앞서 사용된 분류 모델에 위치추정 기능을 갖는 출력층을 추가하면 됨.
    * 위치추정 출력층: 값예측을 위한 네 개의 뉴런 사용.

```python
base_model = keras.applications.xception.Xception(weights="imagenet",
                                                  include_top=False)
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
class_output = keras.layers.Dense(n_classes, activation="softmax")(avg)
loc_output = keras.layers.Dense(4)(avg)
model = keras.models.Model(inputs=base_model.input,
                           outputs=[class_output, loc_output])
model.compile(loss=["sparse_categorical_crossentropy", "mse"],
              loss_weights=[0.8, 0.2], # 어느 출력에 보다 집중할지 결정
              optimizer=optimizer, metrics=["accuracy"])
```

### 경계상자 레이블링

* 경계상자 추정 모델을 학습하려면 모든 이미지에 경계상자 레이블이 있어야 하거나 추가해야함.

* 하지만 가장 어려우면서 고비용이 요구되는 작업이 선행되어야 함.

#### 경계상자 레이블링 도구 소개

* 오픈소스 프로그램
    * VGG Image Annotation
    * LabelImg
    * OpenLabeler
    * ImgLab

* 상용 프로그램
    * LabelBox
    * Supervisely

* 크라우드소싱(crowdsourcing) 플랫폼
    * Amazon Mechanical Turk
        * 아주 많은 양의 이미지에 경계상자 등 표기할 경우
        * 많은 준비작업이 요구됨.
    * 참조 논문: [Crowdsourcing in Computer Vision, Adriana Kovashka et al.](https://arxiv.org/abs/1611.02145)

#### 이미지 전처리

* 경계상자 레이블링 후 두 개의 레이블을 사용하는 이미지를 전처리해야 함.
    모델에 입력되는 샘플의 형식은 다음과 같음.

    ```python
    (images, (class_labels, bounding_boxes))
    ```

#### 경계상자 평가지표: IoU

* 경계상자 예측값들의 정확도 평가함수

* 전체 영역에서 겹치는 부분의 비율로 평가

* MSE 보다 높은 정확도 보임.

<div align="center"><img src="images/ch14/homl14-28.png" width="400"/></div>

## 객체탐지

### 완전 합성곱신경망

### YOLO

## 의미분할(Semantic Segmentation)